Функция на вход получает список из белков и/или список из болезней и выдает граф. Дефолтный граф - полный.

## Примеры работы plot_graph функции

На вход функции plot_graph подаются аргументы:
- table = tb2 (таблица соответствий белок - заболевание, по которой строится граф)
- proteins = None (интересующие белки)
- diseases = None (интересующие болезни)
- depth = 0 (по чему строится граф - по классам болезней (0) или по болезням (1)
- view = "b" (используемый модуль, b - bokeh, c - ipycytoscape)

In [6]:
plot_graph()

Loading BokehJS ...

При помощи аргументов proteins и diseases можно строить граф по конкретным белкам и конкретным болезням. На вход подается списки из белков, болезней или классов болезней.
Например, если proteins = ["Alpha-1-acid glycoprotein 1", "Alpha-1-antitrypsin", "Collagen alpha-1(III) chain"], а diseases = ["Раны и травмы", "Adverse reaction to drug"], то построенный граф будет только про заданные белки и про заданные болезни.

In [17]:
plot_graph(proteins = ["Alpha-1-acid glycoprotein 1", "Alpha-1-antitrypsin", "Collagen alpha-1(III) chain"], diseases = ["Раны и травмы", "Adverse reaction to drug"] )

Loading BokehJS ...

При помощи аргумента depth (=0 или =1) задается, про что строится граф - про классы болезней или про болезни. Если depth = 1, то граф строится про классы болезней (тех, которые были поданы на вход; если же на вход был подан сразу класс болезни - то про него). 
Например, если depth = 0, diseases = ["Раны и травмы", "Adverse reaction to drug"], то граф будет про классы болезней "Раны и травмы" и "Заболевания вследствие воздействия химических веществ" (так как именно к этому классу относится "Adverse reaction to drug").
Если depth = 1, diseases = ["Раны и травмы", "Adverse reaction to drug"], то граф будет про болезни, относящиеся к классу "Раны и травмы", и про "Adverse reaction to drug".

In [23]:
plot_graph(depth = 0, diseases = ["Раны и травмы", "Adverse reaction to drug"])

Loading BokehJS ...

In [24]:
plot_graph(depth = 1, diseases = ["Раны и травмы", "Adverse reaction to drug"])

Loading BokehJS ...

Функция способна строить один и тот же граф при помощи двух разных модулей.
Модуль bokeh (view = "b"). Чем краснее и больше узел - тем больше с ним связей. При наведении на узел можно увидеть его название (type) и количество связей.
Модуль ipycytoscape (view = "c"). Узлы болезней (или классов болезней) - зеленые, узлы белков - красные.

In [27]:
plot_graph(depth = 0, diseases = ["Раны и травмы", "Adverse reaction to drug"], view = "b")

Loading BokehJS ...

In [28]:
plot_graph(depth = 0, diseases = ["Раны и травмы", "Adverse reaction to drug"], view = "c")

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'font-famil…

## Подготовка 

In [1]:
# загрузка необходимых таблиц

import pandas as pd

tb2 = pd.read_csv("tb2.tsv", sep = "\t")
#edges_df = pd.read_csv("edges_df.tsv", sep = "\t")
dis = pd.read_csv("diseases.tsv", sep = "\t")
#nodes_df = pd.read_csv("C:/Users/a120/MRM/nodes_df.tsv", sep = "\t")

In [2]:
def concat(*args):
    return pd.concat([x for x in args if not x.empty])

In [3]:
# загрузка необходимых пакетов

import pandas as pd
import networkx
import matplotlib.pyplot as plt
import numpy as np
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine
from bokeh.plotting import figure
from bokeh.plotting import from_networkx
from bokeh.plotting import output_file
from bokeh.palettes import Blues8, Reds8, Purples8, Oranges8, Viridis8, Spectral8
from bokeh.transform import linear_cmap

import ipycytoscape
import ipywidgets
import json

## Общая функция

In [4]:
def plot_graph(table = tb2, proteins = None, diseases = None, depth = 0, view = "b"):
    # функция строит граф ассоциации болезнь-белок
    # на вход подается таблица, список белков и список болезней
    # список белков и список болезней - в формате списка []
    # аргумент depth показывает, до какой "глубины" нужно рисовать граф:
    # только про классы или про болезни вообще
    # depth = 0 --> только про классы (если изначально как diseases не введена болезнь)
    # depth = 1 --> про болезни
    # аргумент view показывает, с помощью какого модуля строить граф:
    # "b" --> модуль bokeh
    # "c" --> модуль ipycytoscape
    
    diseases_list = []
    classes_list = []
    table1 = pd.DataFrame()
    table2 = pd.DataFrame()
    
    if diseases != None:
        for item in diseases:
            if item in list(dis.diseaseName):
                diseases_list.append(item)
            if item in list(dis.disease):
                classes_list.append(item)
    
    if depth == 0:
        if proteins != None:
            proteins = pd.DataFrame(proteins)
            proteins.columns = ["Protein"]
            table = table.merge(proteins, on = "Protein")
        
        if len(diseases_list) != 0:
            diseases_list = pd.DataFrame(diseases_list)
            diseases_list.columns = ["diseaseName"]
            table1 = table.merge(diseases_list, on = "diseaseName")
            
        if len(classes_list) != 0:
            classes_list = pd.DataFrame(classes_list)
            classes_list.columns = ["disease"]
            table2 = table.merge(classes_list, on = "disease")
            
        if not table1.empty or not table2.empty:
            table = concat(*[table1, table2])
            table = table.drop_duplicates()
        # собственно, создаем граф    
        G = networkx.from_pandas_edgelist(table, "Protein", "disease")
            
    elif depth == 1:
        if proteins != None:
            proteins = pd.DataFrame(proteins)
            proteins.columns = ["Protein"]
            table = table.merge(proteins, on = "Protein")
        if len(diseases_list) != 0:
            diseases_list = pd.DataFrame(diseases_list)
            diseases_list.columns = ["diseaseName"]
            table1 = table.merge(diseases_list, on = "diseaseName")
        if len(classes_list) != 0:
            classes_list = pd.DataFrame(classes_list)
            classes_list.columns = ["disease"]
            table2 = table.merge(classes_list, on = "disease")
        if not table1.empty or not table2.empty:
            table = concat(*[table1, table2])
            table = table.drop_duplicates()
        # собственно, создаем граф
        G = networkx.from_pandas_edgelist(table, "Protein", "diseaseName")
    
    if view == "b":
        # аутпут в ноутбук
        output_notebook()

        # подгонка размера и цвета
        degrees = dict(networkx.degree(G))
        networkx.set_node_attributes(G, name='degree', values=degrees)
        number_to_adjust_by = 5
        adjusted_node_size = dict([(node, 0.3*degree+number_to_adjust_by) for node, degree in networkx.degree(G)])
        networkx.set_node_attributes(G, name='adjusted_node_size', values=adjusted_node_size)
        size_by_this_attribute = 'adjusted_node_size'
        color_by_this_attribute = 'adjusted_node_size'

        # Blues8, Reds8, Purples8, Oranges8, Viridis8
        color_palette = Reds8

        title = 'Protein-disease associations'

        # hovering
        HOVER_TOOLTIPS = [("Type", "@index"), ("Degree", "@degree")]

        plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title)

        # network graph object
        network_graph = from_networkx(G, networkx.spring_layout, scale=10, center=(0, 0))

        # node sizes and colors according to node degree
        minimum_value_color = max(network_graph.node_renderer.data_source.data[color_by_this_attribute])
        maximum_value_color = min(network_graph.node_renderer.data_source.data[color_by_this_attribute])
        network_graph.node_renderer.glyph = Circle(size=size_by_this_attribute, fill_color=linear_cmap(color_by_this_attribute, color_palette, minimum_value_color, maximum_value_color))

        # edge opacity and width
        network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)

        plot.renderers.append(network_graph)

        show(plot)
        
    if view == "c":
        classes_list = list(set(table["disease"]))
        diseases_list= list(set(table["diseaseName"]))
        common_list = classes_list+diseases_list

        my_style = [{'selector': 'node','style': {'font-family': 'arial','font-size': '10px','label': 'data(id)','background-color': 'red'}}]

        for item in common_list:
            text = "node[id = \"" + item + "\"]"
            my_style.append({'selector': text,'style': {'font-family': 'arial','font-size': '10px','label': item,'background-color': 'green'}})

        ipycytoscape_obj = ipycytoscape.CytoscapeWidget()
        ipycytoscape_obj.graph.add_graph_from_networkx(G)
        ipycytoscape_obj.set_style(my_style)
        ipycytoscape_obj.set_style(my_style)
        return ipycytoscape_obj

## Функция для bokeh

In [ ]:
def plot_bgraph(table = tb2, proteins = None, diseases = None, depth = 0):
    # функция строит граф ассоциации болезнь-белок
    # на вход подается таблица, список белков и список болезней
    # список белков и список болезней - в формате списка []
    # аргумент deep показывает, до какой "глубины" нужно рисовать граф:
    # только про классы или про болезни вообще
    # depth = 0 --> только про классы (если изначально как diseases не введена болезнь)
    # depth = 1 --> про болезни
    
    diseases_list = []
    classes_list = []
    table1 = pd.DataFrame()
    table2 = pd.DataFrame()
    
    if diseases != None:
        for item in diseases:
            if item in list(dis.diseaseName):
                diseases_list.append(item)
            if item in list(dis.disease):
                classes_list.append(item)
    
    if depth == 0:
        if proteins != None:
            proteins = pd.DataFrame(proteins)
            proteins.columns = ["Protein"]
            table = table.merge(proteins, on = "Protein")
        
        if len(diseases_list) != 0:
            diseases_list = pd.DataFrame(diseases_list)
            diseases_list.columns = ["diseaseName"]
            table1 = table.merge(diseases_list, on = "diseaseName")
            
        if len(classes_list) != 0:
            classes_list = pd.DataFrame(classes_list)
            classes_list.columns = ["disease"]
            table2 = table.merge(classes_list, on = "disease")
            
        if not table1.empty or not table2.empty:
            table = concat(*[table1, table2])
            table = table.drop_duplicates()
        # собственно, создаем граф    
        G = networkx.from_pandas_edgelist(table, "Protein", "disease")
            
    elif depth == 1:
        if proteins != None:
            proteins = pd.DataFrame(proteins)
            proteins.columns = ["Protein"]
            table = table.merge(proteins, on = "Protein")
        if len(diseases_list) != 0:
            diseases_list = pd.DataFrame(diseases_list)
            diseases_list.columns = ["diseaseName"]
            table1 = table.merge(diseases_list, on = "diseaseName")
        if len(classes_list) != 0:
            classes_list = pd.DataFrame(classes_list)
            classes_list.columns = ["disease"]
            table2 = table.merge(classes_list, on = "disease")
        if not table1.empty or not table2.empty:
            table = concat(*[table1, table2])
            table = table.drop_duplicates()
        # собственно, создаем граф
        G = networkx.from_pandas_edgelist(table, "Protein", "diseaseName")
    
    # аутпут в ноутбук
    output_notebook()

    # подгонка размера и цвета
    degrees = dict(networkx.degree(G))
    networkx.set_node_attributes(G, name='degree', values=degrees)
    number_to_adjust_by = 5
    adjusted_node_size = dict([(node, 0.3*degree+number_to_adjust_by) for node, degree in networkx.degree(G)])
    networkx.set_node_attributes(G, name='adjusted_node_size', values=adjusted_node_size)
    size_by_this_attribute = 'adjusted_node_size'
    color_by_this_attribute = 'adjusted_node_size'

    # Blues8, Reds8, Purples8, Oranges8, Viridis8
    color_palette = Reds8

    title = 'Protein-disease associations'

    # hovering
    HOVER_TOOLTIPS = [("Type", "@index"), ("Degree", "@degree")]

    plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title)

    # network graph object
    network_graph = from_networkx(G, networkx.spring_layout, scale=10, center=(0, 0))

    # node sizes and colors according to node degree
    minimum_value_color = max(network_graph.node_renderer.data_source.data[color_by_this_attribute])
    maximum_value_color = min(network_graph.node_renderer.data_source.data[color_by_this_attribute])
    network_graph.node_renderer.glyph = Circle(size=size_by_this_attribute, fill_color=linear_cmap(color_by_this_attribute, color_palette, minimum_value_color, maximum_value_color))

    # edge opacity and width
    network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)

    plot.renderers.append(network_graph)

    show(plot)

## Функция для ipycytoscape 

In [ ]:
def plot_cgraph(table = tb2, proteins = None, diseases = None, depth = 0):
    
    # функция строит граф ассоциация болезнь-белок
    # на вход подается таблица связей, таблица узлов, список белков и список болезней
    # список белков и список болезней - в формате списка []
    
    diseases_list = []
    classes_list = []
    table1 = pd.DataFrame()
    table2 = pd.DataFrame()
    
    if diseases != None:
        for item in diseases:
            if item in list(dis.diseaseName):
                diseases_list.append(item)
            if item in list(dis.disease):
                classes_list.append(item)
    
    if depth == 0:
        if proteins != None:
            proteins = pd.DataFrame(proteins)
            proteins.columns = ["Protein"]
            table = table.merge(proteins, on = "Protein")
        
        if len(diseases_list) != 0:
            diseases_list = pd.DataFrame(diseases_list)
            diseases_list.columns = ["diseaseName"]
            table1 = table.merge(diseases_list, on = "diseaseName")
            
        if len(classes_list) != 0:
            classes_list = pd.DataFrame(classes_list)
            classes_list.columns = ["disease"]
            table2 = table.merge(classes_list, on = "disease")

        if not table1.empty or not table2.empty:
            table = concat(*[table1, table2])
            table = table.drop_duplicates()
        # собственно, создаем граф    
        G = networkx.from_pandas_edgelist(table, "Protein", "disease")
            
    elif depth == 1:
        if proteins != None:
            proteins = pd.DataFrame(proteins)
            proteins.columns = ["Protein"]
            table = table.merge(proteins, on = "Protein")
        if len(diseases_list) != 0:
            diseases_list = pd.DataFrame(diseases_list)
            diseases_list.columns = ["diseaseName"]
            table1 = table.merge(diseases_list, on = "diseaseName")
        if len(classes_list) != 0:
            classes_list = pd.DataFrame(classes_list)
            classes_list.columns = ["disease"]
            table2 = table.merge(classes_list, on = "disease")
        if not table1.empty or not table2.empty:
            table = concat(*[table1, table2])
            table = table.drop_duplicates()
        # собственно, создаем граф
        G = networkx.from_pandas_edgelist(table, "Protein", "diseaseName")
        
    classes_list = list(set(tb2["disease"]))
    diseases_list= list(set(tb2["diseaseName"]))
    common_list = classes_list+diseases_list

    my_style = [{'selector': 'node','style': {'font-family': 'arial','font-size': '10px','label': 'data(id)','background-color': 'red'}}]

    for item in common_list:
        text = "node[id = \"" + item + "\"]"
        my_style.append({'selector': text,'style': {'font-family': 'arial','font-size': '10px','label': item,'background-color': 'green'}})

    ipycytoscape_obj = ipycytoscape.CytoscapeWidget()
    ipycytoscape_obj.graph.add_graph_from_networkx(G)
    ipycytoscape_obj.set_style(my_style)
    ipycytoscape_obj.set_style(my_style)
    return ipycytoscape_obj